## Data Wrangling and Feature Engineering Exercise

Place your answers within the code blocks that have the comment.

```python
# your code here
```

Make sure to remove or comment the line below to be able to proceed. It's only placed there as a reminder that the function has not yet been implemented.

```python
raise NotImplementedError
```

## Note on "test scripts"

The code bloock following the function definition are "unit tests" which are code blocks which test if the function you implemented is correct or not. It will not print an output **if the conditions are met** (meaning the answer is correct).

### Boston Dataset

We will be using the boston house pricing dataset for exercises 1-5.

In [ ]:
import pandas as pd
from sklearn.datasets import load_boston

data_bunch = load_boston()
data = data_bunch['data']
feature_names = data_bunch['feature_names']
descr = data_bunch['DESCR']

df = pd.DataFrame(data, columns=feature_names)
df.head()

In [ ]:
df.dtypes

In [ ]:
print(descr)

**Exercise 1.** Change the data type of `CHAS` column to **int** and apply one-hot encoding using `pandas`. **(2 pts.)**

In [ ]:
df1 = df.copy()
df1["CHAS"] = df1["CHAS"].astype(int)
df1_onehot = pd.get_dummies(df1, columns=["CHAS"])
df1_onehot.head()

In [ ]:
df1_onehot.columns.tolist()

In [ ]:
df1_onehot['CHAS_0'].values[150:170].tolist()

In [ ]:
df1_onehot['CHAS_1'].values[150:170].tolist()

In [ ]:
assert df1.columns.tolist()==['CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 
                              'TAX', 'PTRATIO', 'B', 'LSTAT', 'CHAS_0', 'CHAS_1']
assert df1['CHAS_0'].values[150:170].tolist()==[1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1]
assert df1['CHAS_1'].values[150:170].tolist()==[0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0]

**Exercise 2.** Filter out the entries wherein the crime rate is lower than the mean. **(2 pts.)**

In [ ]:
df2 = df.copy()
df2_crim = df2[df2["CRIM"] < df2["CRIM"].mean()] #lower than the mean, the assert is a result of higher than the mean
df2_crim

In [ ]:
df2_crim.shape

In [ ]:
df2_crim.values[0].tolist()

In [ ]:
assert df2.shape==(128, 13)
assert df2.values[0].tolist()==[4.0974, 0.0, 19.58, 0.0, 0.871, 5.468, 100.0, 1.4118, 5.0, 403.0, 14.7, 396.9, 26.42]

**Exercise 3.** Change the values of `RAD` column based on the following rules: **(4 pts.)**
 - if 1 to 4, change it to 1.
 - if 5 to 8, change it to 2.
 - otherwise, change it to 3.

In [ ]:
df3 = df.copy()
df3.loc[(df["RAD"] >= 1) & (df["RAD"] <= 4), "RAD"]  = 1
df3.loc[(df["RAD"] >= 5) & (df["RAD"] <= 8), "RAD"]  = 2
df3.loc[(df["RAD"] > 8) | (df["RAD"] < 0), "RAD"]  = 3
df3

In [ ]:
df3['RAD'].value_counts().to_dict()

In [ ]:
assert df3['RAD'].value_counts().to_dict()=={1: 192, 2: 182, 3: 132}

**Exercise 4.** Using the dataframe output from exercise 3, group the entries based on `RAD` column, and get the mean of crime rates for each group. You should end up with a `Series` data structure. Please refer to the assert to have an idea of what is expected. **(2 pts.)**
 

In [ ]:
df4 = df3.copy()
df4.groupby("RAD")["CRIM"].mean()

In [ ]:
assert df4.to_dict()=={1: 0.2591065625, 2: 0.5190552747252744, 3: 12.759290909090915}

**Exercise 5.** Using the dataframe output from exercise 3, group the entries based on `RAD` column, and get the following statistics: count, mean, std, min, 25%, 50%, 75%, and max crime rates for each group. Have all those statistics as column names for the columns of the new `DataFrame`. This means your DataFrame will have the following columns: **count mean std min 25% 50% 75% max**. Please refer to the assert to have an idea of what is expected. 
**(2 pts.)**
 

In [ ]:
df5 = df3.copy()
df5_rad = df5.groupby("RAD")["CRIM"].describe()
df5_rad

In [ ]:
assert df5.to_dict()=={'count': {1: 192.0, 2: 182.0, 3: 132.0},
 'mean': {1: 0.2591065625, 2: 0.5190552747252748, 3: 12.75929090909091},
 'std': {1: 0.3799485400326451, 2: 0.7861597586426312, 3: 13.0411694541406},
 'min': {1: 0.00632, 2: 0.01311, 3: 2.37857},
 '25%': {1: 0.045107499999999995, 2: 0.087375, 3: 5.6863075},
 '50%': {1: 0.094515, 2: 0.17152499999999998, 3: 9.084990000000001},
 '75%': {1: 0.2744225, 2: 0.5253425, 3: 14.3337},
 'max': {1: 2.63548, 2: 4.0974, 3: 88.9762}}

### Titanic Dataset

For the following items, we will use the titanic dataset. For more details regarding the dataset, see this link: https://www.kaggle.com/c/titanic/data

In [ ]:
df_titanic = pd.read_csv('titanic.csv')
df_titanic.head()

**Exercise 6.** Let's do some feature engineering on the `Name` column, which has the following format:
```<Surname>, <Title> <Name> <Middle Name>```.

Your task is to transform the whole text to just the **Surname**. For example, ***Braund*, Mr. Owen Harris** will become ***Braund*** and ***Heikkinen*, Miss. Laina** will become ***Heikkinen***.

Hint! Use the built-in `split` function in string data types.

**(3 pts.)**
 

In [ ]:
df6 = df_titanic.copy()
name = df6["Name"].str.split(",", n = 1, expand = True)
df6["Name"] = name[0]
df6

In [ ]:
assert df6['Name'].tolist()[:10]==['Braund', 'Cumings', 'Heikkinen', 'Futrelle', 'Allen', 
                                   'Moran', 'McCarthy', 'Palsson', 'Johnson', 'Nasser']

**Exercise 7.** Again, the focus is on column `Name`. Let's assume that the following format holds true for all instances:
```<Surname>, <Title> <Name> <Middle Name>```.

Your task is to create new columns named **Surname** and **Title** containing the corresponding extracted details. For example, ***Braund*, *Mr.* Owen Harris** will get us ***Braund*** for the surname and ***Mr.*** for the title, and ***Heikkinen*, *Miss.* Laina** will get us ***Heikkinen*** for the surname and ***Miss.*** for the title.

Hint! Use the built-in `split` function in string data types.

**(3 pts.)**
 

In [ ]:
df7 = df_titanic.copy()
surname = df7["Name"].str.split(",", n = 1, expand = True)
title = surname[1].str.split(" ", n = 2, expand = True)
df7["Surname"] = surname[0]
df7["Title"] = title[1]
df7

In [ ]:
assert df7['Surname'].tolist()[:10]==['Braund', 'Cumings', 'Heikkinen', 'Futrelle', 'Allen', 
                                      'Moran', 'McCarthy', 'Palsson', 'Johnson', 'Nasser']
assert df7['Title'].tolist()[:10]==['Mr.', 'Mrs.', 'Miss.', 'Mrs.', 'Mr.', 'Mr.', 'Mr.', 'Master.', 'Mrs.', 'Mrs.']

**Exercise 8.** Drop all entries with null values, and then get the cabin letter from the `Cabin` column by simply retrieving the first letter in the string. Assign that into a new column named `CabinLetter`. Lastly, filter out the male entries.

**(4 pts.)**
 

In [ ]:
df8 = df_titanic.copy()
#df8_cabin = df8.dropna(subset = ["Cabin"])
df8_cabin = df8.dropna() #drop if any column is missing
df8_cabin["CabinLetter"] = df8_cabin["Cabin"].str[0]
df8_female = df8_cabin[df8_cabin["Sex"] == "female"]
df8_female

In [ ]:
df8_female.shape

In [ ]:
df8_female.head().to_dict()

In [ ]:
assert df8.shape==(88, 13) 
assert df8.head().to_dict()=={'PassengerId': {1: 2, 3: 4, 10: 11, 11: 12, 52: 53},
 'Survived': {1: 1, 3: 1, 10: 1, 11: 1, 52: 1},
 'Pclass': {1: 1, 3: 1, 10: 3, 11: 1, 52: 1},
 'Name': {1: 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)',
  3: 'Futrelle, Mrs. Jacques Heath (Lily May Peel)',
  10: 'Sandstrom, Miss. Marguerite Rut',
  11: 'Bonnell, Miss. Elizabeth',
  52: 'Harper, Mrs. Henry Sleeper (Myna Haxtun)'},
 'Sex': {1: 'female', 3: 'female', 10: 'female', 11: 'female', 52: 'female'},
 'Age': {1: 38.0, 3: 35.0, 10: 4.0, 11: 58.0, 52: 49.0},
 'SibSp': {1: 1, 3: 1, 10: 1, 11: 0, 52: 1},
 'Parch': {1: 0, 3: 0, 10: 1, 11: 0, 52: 0},
 'Ticket': {1: 'PC 17599',
  3: '113803',
  10: 'PP 9549',
  11: '113783',
  52: 'PC 17572'},
 'Fare': {1: 71.2833, 3: 53.1, 10: 16.7, 11: 26.55, 52: 76.7292},
 'Cabin': {1: 'C85', 3: 'C123', 10: 'G6', 11: 'C103', 52: 'D33'},
 'Embarked': {1: 'C', 3: 'S', 10: 'S', 11: 'S', 52: 'C'},
 'CabinLetter': {1: 'C', 3: 'C', 10: 'G', 11: 'C', 52: 'D'}}